The more complex version of this would use this pygithub package as shown in here:
- https://python.langchain.com/docs/integrations/toolkits/github

But let's try a simpler version that just sets up a bunch of python functions to automate different github actions like add, commit and pull requests.

Let's start with a simple commit to a branch of some repository.

First, I'll create a github repository using some simple commands.

In [ ]:
# git init -b main

# git add . && git commit -m "Some commit"

# gh repo create

Now, let's try asking an agent to write come code and commit the resulting code to the current branch of this repository.

To do that, let's give the agent the necessary tools it will need which in this case will be python functions that perform different github actions using the `subprocess` package.

We'll follow the basic steps for building a langchain agent:

# Steps for building a simple agent:
- Set up the LLM
- Define the tool or toolkit (list of tools)
- Set up a prompt template
- Connect llm with your tools
- Define your agent as a dict with keys: input, and agent scratchpad 
- Use the Langchain LCEL language to pipe agent, prompt the llm_with_tools variable and the output parser (which can use OpenAI function calling parser)
- Create the agent loop
- Wrap everything into the AgentExecutor
- Invoke the agent with some query input


In [3]:
# setup the llm
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0)

Now, let's create a tool for the LLM

In [2]:
import subprocess

def github_commit_tool(commit_message="Some commit"):
    subprocess.run(["git", "add", "."])
    subprocess.run(["git", "commit", "-m", commit_message])
    
    return "Committed to Github"

Now! Before we use it with langchain, let's test it by itself.

In [ ]:
from langchain.tools import tool


In [ ]:





# define tool
from langchain.tools import tool
import os
import nbformat




@tool
def turn_jupyter_notebook_into_python_script(jupyter_notebook_path: str) -> None:
        """Turn a jupyter notebook into a python script or a text file"""
        # Load the notebook
        output_file_path="./output_notebook.txt" 
        file_type='txt'
        with open(jupyter_notebook_path) as f:
            nb = nbformat.read(f, as_version=4)
        
        # Create the output directory if it doesn't exist
        os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
        
        # Convert the notebook to a Python script or a text file
        if file_type == 'py':
            nbformat.write(nb, output_file_path)
        elif file_type == 'txt':
            with open(output_file_path, 'w') as f:
                for cell in nb.cells:
                    if cell.cell_type == 'code':
                        f.write(cell.source + '\n\n')
                    elif cell.cell_type == 'markdown':
                        f.write(cell.source + '\n\n')
        else:
            raise ValueError('Invalid file type. Must be "py" or "txt".')
        
        
tools = [turn_jupyter_notebook_into_python_script]




# setup prompt template

from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are very powerful assistant that helps me write reports for machine learning related tasks."),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])




from langchain.tools.render import format_tool_to_openai_function




# connect llm with your tools

llm_with_tools = llm.bind(functions=[format_tool_to_openai_function(t) for t in tools])




from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser




agent = {"input": lambda x: x["input"],
         "agent_scratchpad": lambda x: format_to_openai_functions(x["intermediate_steps"]),
         } | prompt | llm_with_tools | OpenAIFunctionsAgentOutputParser() 




agent.invoke({
    "input": "I want to write a report from this notebook I've written here: ./reporting_agent_langchain.ipynb",
    "intermediate_steps": []})




from langchain.schema.agent import AgentFinish